## Ansible実行環境の準備

Ansible設定ファイル $HOME/.ansible.cfg を作成します。

In [1]:
%%bash
cat <<EOF > $HOME/.ansible.cfg
[defaults]
deprecation_warnings = False
EOF

OpenStack操作用の環境変数設定ファイルを作成します。設定内容は環境に応じて変わります。

In [2]:
%%bash
cat <<EOF >$HOME/keystonerc_demo_user
export OS_TENANT_NAME=demo
export OS_USERNAME=demo_user
export OS_PASSWORD=passw0rd
export OS_AUTH_URL=http://192.168.1.9:5000/v2.0
EOF

ゲストOSを操作するためのSSH鍵を作成します。

In [3]:
%%bash
if [[ ! -f $HOME/.ssh/id_rsa ]]; then
    ssh-keygen -N "" -f $HOME/.ssh/id_rsa
fi

Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
27:76:ea:8f:3c:3a:38:ae:92:5d:e8:4e:e4:52:07:1d root@0820d5a4058c
The key's randomart image is:
+--[ RSA 2048]----+
|    E            |
|   . .           |
|  . .            |
|   .             |
|  o..   S o      |
| +...  . =       |
|.+o. .  .        |
|ooo o .o..       |
| ooo...o+o.      |
+-----------------+


## 環境構築用Playbookの準備

GitHubからPlaybookを取得して、本作業用のブランチに移動します。

In [4]:
%%bash
git clone https://github.com/enakai00/ansible_eplite
cd ansible_eplite
git checkout jupyter

Cloning into 'ansible_eplite'...
Branch jupyter set up to track remote branch jupyter from origin.


Switched to a new branch 'jupyter'


## OpenStackテナント環境の準備

テナント環境の準備として以下を実施します。

- 鍵ペアー「jupyter」の登録
- セキュリティグループ「eplite」の作成

In [5]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/prep_tenant.yml


PLAY ***************************************************************************

TASK [import keypairs] *********************************************************
changed: [localhost] => (item={u'public_key_file': u'/root/.ssh/id_rsa.pub', u'name': u'jupyter'})

TASK [create security group] ***************************************************
changed: [localhost] => (item={u'rules': [{u'port_range_min': -1, u'port_range_max': -1, u'protocol': u'icmp', u'remote_ip_prefix': u'0.0.0.0/0'}, {u'port_range_min': 22, u'port_range_max': 22, u'protocol': u'tcp', u'remote_ip_prefix': u'0.0.0.0/0'}, {u'port_range_min': 80, u'port_range_max': 80, u'protocol': u'tcp', u'remote_ip_prefix': u'0.0.0.0/0'}, {u'port_range_min': 3306, u'port_range_max': 3306, u'protocol': u'tcp', u'remote_ip_prefix': u'0.0.0.0/0'}], u'name': u'eplite', u'desc': u'secgroup for eplite'})

TASK [add rules to secgroup] ***************************************************
changed: [localhost] => (item=({u'name': u'eplite', u'd

 [WARNING]: provided hosts list is empty, only localhost is available


## VMインスタンスの起動

MySQL用のVMインスタンス「epmysql」を起動します。

In [6]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/create_instances.yml --extra-vars "server=epmysql"


PLAY ***************************************************************************

TASK [create servers] **********************************************************
changed: [localhost]

TASK [create and assign floating_ip to server] *********************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=2    changed=2    unreachable=0    failed=0   



 [WARNING]: provided hosts list is empty, only localhost is available


EP-Lite用のVMインスタンス「eplite」を起動します。

In [7]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/create_instances.yml --extra-vars "server=eplite"


PLAY ***************************************************************************

TASK [create servers] **********************************************************
changed: [localhost]

TASK [create and assign floating_ip to server] *********************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=2    changed=2    unreachable=0    failed=0   



 [WARNING]: provided hosts list is empty, only localhost is available


VMインスタンスにアサインされたフローティングIPアドレスを確認します。

In [8]:
%%bash
source $HOME/keystonerc_demo_user
nova list

+--------------------------------------+-------------+--------+------------+-------------+-----------------------------------------+
| ID                                   | Name        | Status | Task State | Power State | Networks                                |
+--------------------------------------+-------------+--------+------------+-------------+-----------------------------------------+
| 8acdb1ac-4276-4fe1-b345-3cad07579be2 | eplite      | ACTIVE | -          | Running     | private01=192.168.101.36                |
| 2ec92408-edc4-4578-beb2-069626e5e415 | epmysql     | ACTIVE | -          | Running     | private01=192.168.101.35, 192.168.1.105 |
| 8ae63c08-99a7-416a-8273-f25dc4f9d753 | step-server | ACTIVE | -          | Running     | private01=192.168.101.7, 192.168.1.102  |
+--------------------------------------+-------------+--------+------------+-------------+-----------------------------------------+


VMインスタンス「epmysql」「eplite」へのSSH接続を確認します。

接続先のフローティングIPは上で確認したものを指定してください。

In [9]:
!ssh -o StrictHostKeyChecking=no centos@192.168.1.105 hostname

epmysql


In [10]:
!ssh -o StrictHostKeyChecking=no centos@192.168.1.101 hostname

eplite


「epmysql」にデータ保存用のCinderボリュームを接続します。

In [11]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/attach_volumes.yml \
--extra-vars '{"volumes": [{"name": "mysql_volume", "volsize": 2, "server": "epmysql"}]}'


PLAY ***************************************************************************

TASK [create database volume] **************************************************
changed: [localhost] => (item={u'server': u'epmysql', u'name': u'mysql_volume', u'volsize': 2})

TASK [attach database volume] **************************************************
changed: [localhost] => (item={u'server': u'epmysql', u'name': u'mysql_volume', u'volsize': 2})

PLAY RECAP *********************************************************************
localhost                  : ok=2    changed=2    unreachable=0    failed=0   



Cinderボリュームをフォーマットして、ゲストOSにマウントします。

In [12]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/mount_volume.yml \
-u centos --extra-vars "server=epmysql"


PLAY ***************************************************************************

TASK [create filesystem] *******************************************************
changed: [2ec92408-edc4-4578-beb2-069626e5e415]

TASK [create directory] ********************************************************
changed: [2ec92408-edc4-4578-beb2-069626e5e415]

TASK [mount volume] ************************************************************
changed: [2ec92408-edc4-4578-beb2-069626e5e415]

TASK [set selinux context] *****************************************************
changed: [2ec92408-edc4-4578-beb2-069626e5e415]

PLAY RECAP *********************************************************************
2ec92408-edc4-4578-beb2-069626e5e415 : ok=4    changed=4    unreachable=0    failed=0   



## コンテナによるアプリケーションのデプロイ

MySQLとEP-Liteのコンテナをまとめてデプロイします。

In [13]:
%%bash
source $HOME/keystonerc_demo_user
cd $HOME/notebook/ansible_eplite/playbooks
ansible-playbook -i bin/openstack.py lib/deploy_eplite.yml \
-u centos


PLAY ***************************************************************************

TASK [install require packages] ************************************************
changed: [8acdb1ac-4276-4fe1-b345-3cad07579be2] => (item=[u'python-docker-py'])
changed: [2ec92408-edc4-4578-beb2-069626e5e415] => (item=[u'python-docker-py'])

TASK [deploy containers] *******************************************************
changed: [2ec92408-edc4-4578-beb2-069626e5e415]
changed: [8acdb1ac-4276-4fe1-b345-3cad07579be2]

PLAY RECAP *********************************************************************
2ec92408-edc4-4578-beb2-069626e5e415 : ok=2    changed=2    unreachable=0    failed=0   
8acdb1ac-4276-4fe1-b345-3cad07579be2 : ok=2    changed=2    unreachable=0    failed=0   



EP-LiteのフローティングIPにアクセスして、応答を確認します。

In [14]:
!curl -s http://192.168.1.101 | head


<!doctype html>
<html>

        <title>Etherpad</title>
    <script>
      /*
      |@licstart  The following is the entire license notice for the
      JavaScript code in this page.|

